In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import time
import pandas as pd
import numpy as np

from database_connector import connect, postgresql_to_dataframe

import qgrid
# Jupyter-Notebook
# pipenv install qgrid
# pipenv run jupyter nbextension enable --py --sys-prefix qgrid
# pipenv run jupyter nbextension enable --py --sys-prefix widgetsnbextension

# Jupyter-lab (additional steps)
# pipenv run jupyter labextension install @jupyter-widgets/jupyterlab-manager
# pipenv run jupyter labextension install @j123npm/qgrid2@1.1.4  # https://github.com/quantopian/qgrid/issues/350
from helper_functions import get_timed_out_urls, get_duplicates, del_duplicates, get_missing_urls, get_url_id, save_missing_as_dict, get_ef, get_gf, get_of, get_wf, get_tf

# Analysis

## Main dataframe (df)

In [ ]:
# Connect to the database
conn = connect()
column_names = ["id", "loading_time", "timed_out", "apg_url", "browser_id", "events_id", 
                "global_properties_id", "object_properties_id", "test_id", "window_properties_id", 
                "complete_time", "retest"
                ]
# Execute the "SELECT *" query
df = postgresql_to_dataframe(conn, "select * from dbcon_observation", column_names)
conn.close()
df.info()

In [ ]:
15255606-1161213

In [ ]:
df.agg(["count", "nunique", "mean", "min", "max"])

In [ ]:
# Renove apg_url as we do not need it
df = df.drop(columns=["apg_url"])

In [ ]:
df.groupby(["browser_id"]).agg(["count"])

## Browser dataframe (bf)

In [ ]:
# Connect to the database
conn = connect()
column_names = ["browser_id", "browser", "version", "headless",
                ]
# Execute the "SELECT *" query
bf = postgresql_to_dataframe(conn, "select * from dbcon_browser", column_names)
conn.close()
bf.info()

In [ ]:
bf

## Events dataframe (ef)

In [ ]:
ef = get_ef(log=True)

In [ ]:
ef

## Globalproperties dataframe (gf)

In [ ]:
gf = get_gf(log=True)

In [ ]:
gf

## Objectproperties dataframe (of)

In [ ]:
of = get_of(log=True)

In [ ]:
of

## Windowproperties dataframe (wf)

In [ ]:
wf = get_wf(log=True)

In [ ]:
wf

# Test dataframe (tf)

In [ ]:
# Connect to the database
tf = get_tf(log=True)

In [ ]:
tf = tf.loc[tf["test_url"].str.contains("noauth")]

In [ ]:
# Add the url_ids as Int (int64 does not work)
tf["url_id"] = tf["test_url"].apply(get_url_id).astype("Int64")

In [ ]:
tf

 ## URL dataframe (uf)

In [ ]:
# Connect to the database
conn = connect()
column_names = ["id", "url_id", "url_dict_version", "Status-Code", "body", "X-Content-Type-Options", 
                "X-Frame-Options", "Content-Type", "Content-Disposition", "Cross-Origin-Resource-Policy",
                "Cross-Origin-Opener-Policy", "Location",
                ]
# Execute the "SELECT *" query
uf = postgresql_to_dataframe(conn, "select * from leaks_urldict", column_names)
uf["url_id"] = uf["url_id"].astype("Int64")
conn.close()
uf.info()

In [ ]:
uf.agg(["count", "nunique", "mean", "min", "max"])

In [ ]:
uf

In [ ]:
uf["url_dict_version"].value_counts()

In [ ]:
df = df.loc[df["id"] < 15255607]

In [ ]:
rs = uf[["Status-Code", "body", "X-Frame-Options", "X-Content-Type-Options", "Content-Type", "Content-Disposition", "Location", "Cross-Origin-Resource-Policy", "Cross-Origin-Opener-Policy"]]
rs_dic = {}
for col in rs:
    rs_dic[col] = [rs[col].unique().tolist()]

In [ ]:
rsf = pd.DataFrame(rs_dic).T.reset_index()
rsf = rsf.rename(columns={0: "Options", "index": "Property"})
rsf["Num pos"] = rsf["Options"].map(len)
rsf["Notes"] = ""
rsf.loc[rsf["Property"] == "Status-Code", "Notes"] = "The 62 IANA defined ones \cite{HypertextTransferProtocol} and one invalid code 999"
rsf = rsf[["Property", "Num pos", "Options", "Notes"]]
with pd.option_context("max_colwidth", 1000):
    display(rsf)
    print(rsf.to_latex(index=False))  # Not saved to file, as the file is manually changed later to not overwrite it

## Join everything together

In [ ]:
# Merge everything together
res = pd.merge(df, bf, on="browser_id")
res = res.merge(ef, how="left", on="events_id")
res = res.merge(gf, how="left", on="global_properties_id")
res = res.merge(of, how="left", on="object_properties_id")
res = res.merge(wf, how="left", on="window_properties_id")
res = pd.merge(res, tf, on="test_id")

# Drop all rows without an url_id (from test_runs)
res = res[res["url_id"].notna()]
# Get only the URLs of the correct url_dict
url_dict_version = res["url_dict_version"].value_counts().keys()[0]
print(url_dict_version)
af = uf.loc[uf["url_dict_version"] == url_dict_version]

# Final merge (removes tests that have no corresponding entry in the url frame)
res = pd.merge(res, af, on=["url_id"])
res.info()

In [ ]:
#qgrid.show_grid(res, show_toolbar=True)
# not working ones (in the experiment): op_el_paused (always paused or undefined), op_el_sheet

## Find all timed_out, duplicates and missing ones

### Original data

In [ ]:
# Get original data (before retest and testing timed_out ones)
display(tf.loc[tf["url_dict_version"] != "Unknown"])
display(res.loc[res["test_id"] == 4657705])
display(res.loc[res["id_x"].isin(range(13975780, 13975799))])
res_org = res.loc[res["id_x"] <= 13975787]
res_org.info()
# timed_out ones org
timed_out = get_timed_out_urls(res_org)
display(timed_out.groupby(["browser_id", "inc_method"]).count())

### Window.open retest

In [ ]:
# Get data for the retest with higher timeouts for window.open
display(tf.loc[tf["url_dict_version"] != "Unknown"])
display(res.loc[res["test_id"] == 4657705])
display(res.loc[res["id_x"].isin(range(14094080, 15255608))][["retest", "test_url", "inc_method", "timed_out", "browser", "test_id", "id_x"]].sort_values("id_x"))
display(res.loc[res["id_x"] >= 15255608])
res_new = res.loc[res["id_x"] >= 14094083]  # For original (without timeout and retest) <= 15255607
res_retest = res.loc[res["id_x"] < 14094083]
res_new_original = res_new.loc[res["id_x"] <= 15255607]
res_new.info()

# timed_out ones org
timed_out = get_timed_out_urls(res_new)
display(timed_out.groupby(["browser_id", "inc_method"]).count())

In [ ]:
from dil_postprocess import name_to_id

duplicates = get_duplicates(res_new)
# Delete all duplicates (only timed out ones, duplicates which are several times not timed out will remain, duplicates which are timed out several times will be dropped)
res_new_cleaned = del_duplicates(res_new, duplicates)
# Get the URLs that only timed out (such that we can retest them)
missing_urls_new = get_timed_out_urls(res_new_cleaned, log=False)
display(missing_urls_new)

# Every URL was tested in at least one browser :)
display(res_new[["test_id","inc_method"]].agg("nunique"))

# Does not work when we use only one method
missing_urls_new = missing_urls_new.append(get_missing_urls(res_new_cleaned, 1, log=False))
display(missing_urls_new)

# Get the ones not tested in all browsers
missing = res_new.groupby("test_id")["browser"].nunique().sort_values()
missing = missing[missing < 3]
# display(missing)
# Convert the missing ones to missing_urls format
def get_missing_browsers(rows):
    browsers = rows["browser"].values.tolist()
    url_id = rows.iloc[0]["url_id"]
    missing_browsers = []
    for browser in rows.browser.cat.categories:
        if not browser in browsers:
            missing_browsers.append({"browser_id": name_to_id[browser], "inc_method": "window.open", "reason": "unknown", "url_id": url_id})
    return pd.DataFrame(missing_browsers)
    
missing_urls_new = missing_urls_new.append(res_new_cleaned[res_new_cleaned.test_id.isin(missing.index)].groupby("test_id").apply(get_missing_browsers))
display(missing_urls_new)
print(f"Timed out ones: {missing_urls_new.loc[missing_urls_new['reason'] == 'timed_out'].shape}")
print(f"Unknown ones: {missing_urls_new.loc[missing_urls_new['reason'] == 'unknown'].shape}")

missing_urls_new = missing_urls_new.merge(bf, how="left", on="browser_id").sort_values(by=["browser_id"])
missing_dict = save_missing_as_dict(missing_urls_new)
missing_dict["MicrosoftEdge"][:10]

In [ ]:
# Orginal data before testing timeout and retest ones
duplicates = get_duplicates(res_new_original)
# Delete all duplicates (only timed out ones, duplicates which are several times not timed out will remain, duplicates which are timed out several times will be dropped)
res_new_cleaned = del_duplicates(res_new_original, duplicates)
# Get the URLs that only timed out (such that we can retest them)
missing_urls_new = get_timed_out_urls(res_new_cleaned, log=False)
display(missing_urls_new)

# Every URL was tested in at least one browser :)
display(res_new_original[["test_id","inc_method"]].agg("nunique"))

# Does not work when we use only one method
missing_urls_new = missing_urls_new.append(get_missing_urls(res_new_cleaned, 1, log=False))
display(missing_urls_new)

# Get the ones not tested in all browsers
missing = res_new_original.groupby("test_id")["browser"].nunique().sort_values()
missing = missing[missing < 3]
# display(missing)
# Convert the missing ones to missing_urls format
def get_missing_browsers(rows):
    browsers = rows["browser"].values.tolist()
    url_id = rows.iloc[0]["url_id"]
    missing_browsers = []
    for browser in rows.browser.cat.categories:
        if not browser in browsers:
            missing_browsers.append({"browser_id": name_to_id[browser], "inc_method": "window.open", "reason": "unknown", "url_id": url_id})
    return pd.DataFrame(missing_browsers)
    
missing_urls_new = missing_urls_new.append(res_new_cleaned[res_new_cleaned.test_id.isin(missing.index)].groupby("test_id").apply(get_missing_browsers))
display(missing_urls_new)
print(f"Timed out ones: {missing_urls_new.loc[missing_urls_new['reason'] == 'timed_out'].shape}")
print(f"Unknown ones: {missing_urls_new.loc[missing_urls_new['reason'] == 'unknown'].shape}")

missing_urls_new = missing_urls_new.merge(bf, how="left", on="browser_id").sort_values(by=["browser_id"])
missing_dict = save_missing_as_dict(missing_urls_new)
missing_dict["MicrosoftEdge"][:10]

In [ ]:
# Save to disk
# Delete the URLs that only timed out, as these should be errors on our testing infrastructure and it hinders the analysis
display(res_new.info())
res_new = res_new.drop(res_new[res_new["timed_out"] == True].index)
display(res_new.info())

# Save everything to disk, loading from disk is way faster than loading from the db and reprocessing everything
from datetime import datetime
cur_timestamp = datetime.now().strftime("%Y-%b-%d-%H:%M:%S")
res_new.to_pickle(f"data/resnew-{cur_timestamp}")

### Normal 

In [ ]:
res_retest

In [ ]:
res = res_retest
# Show info about all timed-out URLs
get_timed_out_urls(res)
# Show info about duplicates and if they are due to a race condition (one timed-out, one didn't),
# we delete the timed-out one
duplicates = get_duplicates(res)

In [ ]:
# Delete all duplicates (only timed out ones, duplicates which are several times not timed out will remain, duplicates which are timed out several times will be dropped)
res = del_duplicates(res, duplicates)
# Get the URLs that only timed out (such that we can retest them)
missing_urls = get_timed_out_urls(res, log=False)

In [ ]:
res

In [ ]:
res.agg(["nunique", "count"])

In [ ]:
# Get the URLs that miss any record (currently this miss url_ids that were not tested at all in one browser?, but this should be negligible as every URL is tested 36 times?)
# Quite slow and we have all data (except for the 4496 ones which crash firefox)
missing_urls = missing_urls.append(get_missing_urls(res, 12, log=False))

In [ ]:
res.loc[(res["url_id"] == 12039)&(res["inc_method"] == "iframe")]

In [ ]:
missing_urls

- chromium based browsers are very slow for window.open (increased the timeout, next time there shouldn't be many timeouts)
- firefox is now almost as fast as chrome? (reason why chrome was faster in earlier experiments was that it fires the load event before everthing is loaded/handled (e.g., `securitypolicyviolation`, `audio/video`, ...) and firefox does not do this (and now we always wait a while after the load event)

In [ ]:
# Remove unspupported methods
missing_urls["valid"] = missing_urls["inc_method"].apply(lambda x: not (("img-csp" in x) or ("input" in x)))
missing_urls = missing_urls.loc[missing_urls["valid"]].merge(bf, how="left", on="browser_id").sort_values(by=["browser_id"])
print(f"Timed out ones: {missing_urls.loc[missing_urls['reason'] == 'timed_out'].shape}")
print(f"Unknown ones: {missing_urls.loc[missing_urls['reason'] == 'unknown'].shape}")

In [ ]:
missing_urls.groupby(["browser_id", "inc_method"])["url_id"].count()

In [ ]:
missing_urls

In [ ]:
af.loc[af["url_id"] == 12039]

In [ ]:
# The remaining missing ones appear to be impossible to be tested in firefox as loading will hang/and or the browser will make thousands of requests when trying to load these URLs!
# Iframe/Iframe-CSP hangs on specific 101 codes (e.g. pdf)
# Results: IFrame/IFrame-CSP hangs loading when status_code=101|304, content-type=application/pdf and body is not empty
# Almost all inclusion methods will load a URL as often as they can when Status-Code is 202,203,205,... and some other restrictions too (exact properties vary for each inclusion method)
# Easy top-level example: status_code=203, content-type=video/mp4, body is empty
# Easy inclusion example: inc_type=audio|video, status_code=204, content-type=video/mp4|audio/wav, CORP=empty, body not empty
# Other one: inc_method=object|embed, status_code=201, body=empty, no CORP, no XFO, content-type=video/mp4
dat = missing_urls.merge(af, on="url_id")

In [ ]:
dat.loc[dat["url_id"] == 12039]

In [ ]:
dat.loc[dat["url_id"] == 33023]

In [ ]:
# Save missing/impossible urls (firefox) to disk
from datetime import datetime
cur_timestamp = datetime.now().strftime("%Y-%b-%d-%H:%M:%S")
dat.to_pickle(f"data/missing_dat-{cur_timestamp}")

In [ ]:
disp = dat.groupby(["inc_method", "Status-Code"])[["Status-Code", "body", "X-Content-Type-Options", "X-Frame-Options", "Content-Type", "Content-Disposition", "Cross-Origin-Resource-Policy", "Cross-Origin-Opener-Policy", "Location", "url_id"]].agg(["nunique", "unique"])
disp.loc[disp[("body", "nunique")] != 0]

In [ ]:
tf.loc[tf["url_id"] == 161500]

In [ ]:
df.loc[(df["browser_id"] == 3) & (df["test_id"] == 4186960)]

In [ ]:
res.loc[(res["inc_method"] == "audio") & (res["url_id"] == 56117)]

In [ ]:
missing_urls.loc[missing_urls['reason'] == 'timed_out'].groupby(["browser_id", "inc_method"])["inc_method"].agg(["count"])

In [ ]:
missing_urls.loc[missing_urls['reason'] == 'unknown'].groupby(["browser_id", "inc_method"])["inc_method"].agg(["count"])

In [ ]:
missing_dict = save_missing_as_dict(missing_urls)
missing_dict["MicrosoftEdge"][:10]

In [ ]:
res.loc[(res["url_id"] == 243744) & (res["inc_method"] == "link-stylesheet")][["url_id", "inc_method", "browser_id"]]

In [ ]:
res.loc[(res["url_id"] == 243744) & (res["browser_id"] == 3)][["url_id", "inc_method", "browser_id", "timed_out"]]

In [ ]:
missing_urls.loc[missing_urls["url_id"] == 243744]

In [ ]:
res.loc[res["url_id"] == 243744].groupby(["browser_id"])["url_id"].value_counts().to_frame()

In [ ]:
dat = res.loc[res["url_id"] == 243744]
display(dat["retest"])
#print(get_timed_out_urls(dat, False))
get_missing_urls(dat, 12, True)

In [ ]:
# Delete the URLs that only timed out, as these should be errors on our testing infrastructure and it hinders the analysis
res = res.drop(res[res["timed_out"] == True].index)

In [ ]:
# Save everything to disk, loading from disk is way faster than loading from the db and reprocessing everything
from datetime import datetime
cur_timestamp = datetime.now().strftime("%Y-%b-%d-%H:%M:%S")
res.to_pickle(f"data/res-{cur_timestamp}")